### Imports

In [ ]:
import requests
import pandas as pd
import os
import numpy as np
import time
import sys

import openmeteo_requests
import requests_cache
from retry_requests import retry

sys.path.append(os.path.abspath('../utils'))
import tinne_utils as tu

## Notebook constants

In [19]:
ITM_DIR = os.path.join(os.getcwd(), '../data/import')

wind_speed_10m_max,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_hours,relative_humidity_2m_mean,relative_humidity_2m_max,relative_humidity_2m_min

# 🌦 Weather Data Variables

## 🌡 Temperature & Humidity  
- **`temperature_2m`** → Air temperature measured **2 meters above ground** (°C).  
- **`apparent_temperature`** → Feels-like temperature, adjusted for **wind and humidity** (°C).  
- **`relative_humidity_2m`** → The **percentage** of moisture in the air at **2 meters height**.  

## 🌧 Precipitation & Snow  
- **`precipitation`** → Total precipitation (rain + snow) in **millimeters (mm)**.  
- **`rain`** → Total **rainfall** (excluding snow) in **mm**.  
- **`showers`** → **Intermittent, intense bursts of rain** that start and stop suddenly.  
- **`snowfall`** → Total **snow accumulation** in **mm**.  
- **`snow_depth`** → The depth of **snow on the ground** in **mm**.  

## ☁️ Weather Conditions  
- **`weathercode`** → A numerical code representing the **weather condition** (e.g., clear, fog,thunderstorms).  


| Weather Code | Description |
|-------------|------------|
| 0  | Clear sky |
| 1  | Mainly clear |
| 2  | Partly cloudy |
| 3  | Overcast |
| 45 | Fog |
| 48 | Depositing rime fog |
| 51 | Light drizzle |
| 53 | Moderate drizzle |
| 55 | Heavy drizzle |
| 56 | Light freezing drizzle |
| 57 | Heavy freezing drizzle |
| 61 | Light rain |
| 63 | Moderate rain |
| 65 | Heavy rain |
| 66 | Light freezing rain |
| 67 | Heavy freezing rain |
| 71 | Light snow |
| 73 | Moderate snow |
| 75 | Heavy snow |
| 77 | Snow grains |
| 80 | Light rain showers |
| 81 | Moderate rain showers |
| 82 | Heavy rain showers |
| 85 | Light snow showers |
| 86 | Heavy snow showers |
| 95 | Thunderstorm |
| 96 | Thunderstorm with lig`time`** → The **timestamp** of the weather measurement.  
- **`timezone`** → The **local timezone** used for time formatting.  

---

### 📌 Notes  
- **Precipitation is measured in millimeters (mm).**  
- **`showers` differ from `rain`** because they are **intermittent and localized**.  
- **Temperature values are in degrees Celsius (°C).**  


### Request historial weather reports through API

In [20]:
# Bee dataset contains US states information. Latitudes and longitudes are used to get weather data from Open Meteo API.
states = pd.read_csv(os.path.join(ITM_DIR, 'us_states_lat_long.csv'))
states.head()

# rename states columns to state latitude and longitude
states.rename(columns={"State": "state", "Latitude": "latitude", "Longitude": "longitude"}, inplace=True)

locations = states[["latitude", "longitude"]].drop_duplicates()

# API is limited in how many requests can be made at once, so we split the locations into smaller chunks
locations_split = np.array_split(locations, 6)

c:\Users\tinne\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
# Fetch weather data for each part with a delay of 15 minutes between requests
weather_data_list = []
for i, loc in enumerate(locations_split):
    print(f"Fetching weather data for part {i + 1}...")
    
    # Fetch weather data for the current part
    weather_data = tu.fetch_weather_data(loc)
    weather_data_list.append(weather_data)
    
    # If not the last part, wait for 15 minutes before the next request
    if i < len(locations_split) - 1:
        print("Waiting for 15 minutes before the next request...")
        time.sleep(900)  # Wait for 15 minutes

# Combine all weather data into one DataFrame
weather_data_combined = pd.concat(weather_data_list, ignore_index=True)


In [ ]:
# save df's csv
# df1.to_csv("data/import/Hourly_Weather_Data_part1.csv", index=False)
# df2.to_csv("data/import/Hourly_Weather_Data_part2.csv", index=False)
# df3.to_csv("data/import/Hourly_Weather_Data_part3.csv", index=False)
# df4.to_csv("data/import/Hourly_Weather_Data_part4.csv", index=False)
# df5.to_csv("data/import/Hourly_Weather_Data_part5.csv", index=False)
# df6.to_csv("data/import/Hourly_Weather_Data_part6.csv", index=False)